In [1]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
mpl.style.use('../../paper.mcstyle')
import matplotlib.cm as cm

    
from scipy.stats import gaussian_kde


import pandas as pd
from sklearn.neighbors import NearestNeighbors

from multiprocessing import cpu_count
from multiprocessing import Pool

import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn

import ili
from ili.dataloaders import NumpyLoader, TorchLoader
from ili.inference import InferenceRunner
from ili.validation.metrics import PosteriorCoverage, PlotSinglePosterior
from CNN import ConvNet
from CASBI.utils import create_template_library as ctl


In [2]:
if __name__ == '__main__':

    data = pd.read_parquet('/export/data/vgiusepp/data/full_dataframe/dataframe/dataframe.parquet')
    data['star_log10mass'] = 10**data['star_log10mass']
    data_mass = data['star_log10mass'].drop_duplicates()
    data_mass = data_mass[data_mass<1.4*1e9]
    
    m_min, m_max = data_mass.min(), data_mass.max()
    alpha = 1.25
   
    mass_name = data[['star_log10mass', 'Galaxy_name', 'infall_time']].drop_duplicates()
    mass_name = mass_name[mass_name['star_log10mass']<1.4*1e9]
    min_feh, max_feh = data['feh'].min(), data['feh'].max() 
    min_ofe, max_ofe = data['ofe'].min(), data['ofe'].max()
    mass_nn = mass_name['star_log10mass'].values.reshape(-1, 1)
    infall_time = mass_name['infall_time'].values.reshape(-1, 1)
    galaxy_name = mass_name['Galaxy_name'].values.reshape(-1, 1)
    
    test_set_sample = 100
    train_set_sample = 1_000

    mass_nn = mass_name['star_log10mass'].values.reshape(-1, 1)
    infall_time = mass_name['infall_time'].values.reshape(-1, 1)
    galaxy_name = mass_name['Galaxy_name'].values.reshape(-1, 1)
    # with Pool(processes=cpu_count()) as p:
    #     result = p.starmap(ctl.gen_real_halo, [[j, galaxy_name, mass_nn, infall_time, m_max, m_min, alpha] for j in range(test_set_sample)]   )
    
    result = ctl.gen_template_library(test_set_sample, galaxy_name, mass_nn, infall_time, m_max, m_min, alpha)

    hist_list_test, params_list_test, galaxy_list_test = zip(*result)

    #create the filter to take only the unique galaxies 
    single_galaxy_test = [arr[0] for arr in galaxy_list_test if arr.size > 0]
    unique_indices_test = list({tuple(arr): i for i, arr in enumerate(map(tuple, single_galaxy_test))}.values())
    print('unique galaxy in the test set that are not empty:', len(unique_indices_test))

    flattened_hist_list_test = [item for i, sublist in enumerate(hist_list_test) if i in unique_indices_test for item in sublist]
    flattened_param_list_test = [item for i, sublist in enumerate(params_list_test) if i in unique_indices_test for item in sublist]
    flattened_hist_list_test =  np.array(flattened_hist_list_test)
    flattened_param_list_test =  np.array(flattened_param_list_test)
    galaxies_test = [set(arr[0]) for arr in galaxy_list_test if arr.size > 0] #list that contains set of names of the galaxy in the test set to compare it with the training set 


    with Pool(processes=cpu_count()) as p:
        result = p.starmap(ctl.gen_real_halo, [[j+test_set_sample, galaxy_name, mass_nn, infall_time, galaxies_test] for j in range(train_set_sample)]   )

    hist_list, params_list, galaxy_list = zip(*result)

    #create the filter to take only the unique galaxies 
    single_galaxy = [arr[0] for arr in galaxy_list if arr.size > 0]
    unique_indices = list({tuple(arr): i for i, arr in enumerate(map(tuple, single_galaxy))}.values())
    flattened_hist_list = [item for i, sublist in enumerate(hist_list) if i in unique_indices for item in sublist]
    flattened_param_list = [item for i, sublist in enumerate(params_list) if i in unique_indices for item in sublist]
    flattened_hist_list =  np.array(flattened_hist_list)
    flattened_param_list =  np.array(flattened_param_list)
    print('unique galaxy in the training set that are not empty:', len(unique_indices))
    
    mask = [flattened_hist_list[:, 1, 0, 0] < np.random.uniform(low=0, high=100, size=len(flattened_hist_list[:, 1, 0, 0])) ][0] #applying a mask to not overfit on high N
    # mask = [flattened_hist_list[:, 1, 0, 0] < 20 ][0] #applying a mask to not overfit on high N
    training_x = flattened_hist_list[mask]
    training_theta = flattened_param_list[mask]


    test_x = torch.log1p(torch.from_numpy(flattened_hist_list_test)).float()
    test_theta = torch.log10(torch.from_numpy(flattened_param_list_test)).float()

    x = torch.log1p(torch.from_numpy(training_x)).float()
    theta = torch.log10(torch.from_numpy(training_theta)).float()
    
    
    gpu_index = 6  # replace with your desired GPU index
    torch.cuda.set_device(gpu_index)
    device = f"cuda:{gpu_index}"
    conditions  = mass_name[['star_log10mass', 'infall_time']]
    minimum_theta = [np.log10(conditions[col].values.min()) for col in conditions.columns]   
    maximum_theta = [np.log10(conditions[col].values.max()) for col in conditions.columns]    

    
    def write_to_yaml(minimum_theta, maximum_theta, device):
        # Load the existing data
        with open('./training.yaml', 'r') as file:
            data = yaml.safe_load(file)
            
        # Update the value
        # data['prior']['args']['low'] = minimum_theta[0]
        # data['prior']['args']['high'] = maximum_theta[0]
        data['device'] = device

        # Write the data back to the file
        with open('./training.yaml', 'w') as file:
            yaml.safe_dump(data, file)
            
    # write_to_yaml(minimum_theta, maximum_theta, device)
    print('write the right prior in the training.yaml file')


NoneNone
NoneNoneNone



NoneNone

NoneNoneNoneNoneNoneNone


None

NoneNone

None
NoneNone


None
NoneNone
None
None


NoneNoneNone


NoneNone

None
None
NoneNone

None
NoneNone
None
NoneNoneNone
None


None
None

NoneNone
NoneNoneNoneNone
NoneNone

None
None



None

NoneNone
None

NoneNoneNone

None

None
None
NoneNone

NoneNone
None

None
None
None
NoneNone

None
None
None
None
None
NoneNone

NoneNone

NoneNone

NoneNone

None
None
None
None
None
NoneNone

None
None
None
None
NoneNone
None

unique galaxy in the test set that are not empty: 96


In [ ]:

    with Pool(processes=cpu_count()) as p:
        result = p.starmap(ctl.gen_real_halo, [[j+test_set_sample, galaxy_name, mass_nn, infall_time, galaxies_test] for j in range(train_set_sample)]   )

    hist_list, params_list, galaxy_list = zip(*result)

    #create the filter to take only the unique galaxies 
    single_galaxy = [arr[0] for arr in galaxy_list if arr.size > 0]
    unique_indices = list({tuple(arr): i for i, arr in enumerate(map(tuple, single_galaxy))}.values())
    flattened_hist_list = [item for i, sublist in enumerate(hist_list) if i in unique_indices for item in sublist]
    flattened_param_list = [item for i, sublist in enumerate(params_list) if i in unique_indices for item in sublist]
    flattened_hist_list =  np.array(flattened_hist_list)
    flattened_param_list =  np.array(flattened_param_list)
    print('unique galaxy in the training set that are not empty:', len(unique_indices))
    
    mask = [flattened_hist_list[:, 1, 0, 0] < np.random.uniform(low=0, high=100, size=len(flattened_hist_list[:, 1, 0, 0])) ][0] #applying a mask to not overfit on high N
    # mask = [flattened_hist_list[:, 1, 0, 0] < 20 ][0] #applying a mask to not overfit on high N
    training_x = flattened_hist_list[mask]
    training_theta = flattened_param_list[mask]


    test_x = torch.log1p(torch.from_numpy(flattened_hist_list_test)).float()
    test_theta = torch.log10(torch.from_numpy(flattened_param_list_test)).float()

    x = torch.log1p(torch.from_numpy(training_x)).float()
    theta = torch.log10(torch.from_numpy(training_theta)).float()
    
    
    gpu_index = 6  # replace with your desired GPU index
    torch.cuda.set_device(gpu_index)
    device = f"cuda:{gpu_index}"
    conditions  = mass_name[['star_log10mass', 'infall_time']]
    minimum_theta = [np.log10(conditions[col].values.min()) for col in conditions.columns]   
    maximum_theta = [np.log10(conditions[col].values.max()) for col in conditions.columns]    

    
    def write_to_yaml(minimum_theta, maximum_theta, device):
        # Load the existing data
        with open('./training.yaml', 'r') as file:
            data = yaml.safe_load(file)
            
        # Update the value
        # data['prior']['args']['low'] = minimum_theta[0]
        # data['prior']['args']['high'] = maximum_theta[0]
        data['device'] = device

        # Write the data back to the file
        with open('./training.yaml', 'w') as file:
            yaml.safe_dump(data, file)
            
    # write_to_yaml(minimum_theta, maximum_theta, device)
    print('write the right prior in the training.yaml file')